<a href="https://colab.research.google.com/github/frozenscar/AutomaticYoutubeShortsGenerator/blob/main/AutoShortsVideoGen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:

import configparser
import os

def create_config(base_path):
    config = configparser.ConfigParser()
    # Add sections and key-value pairs
    config['General'] = {'curr_vid': 0}
    # Write the configuration to a file
    with open(base_path+'/video_config.ini', 'w') as configfile:
        config.write(configfile)

base_path = '/content/drive/MyDrive/AI'
#create_config(base_path)

def set_video(base_path,curr_vid):
  config = configparser.ConfigParser()
  config.read(base_path+'/video_config.ini')
  config['General']['curr_vid'] = str(curr_vid)
  with open(base_path+'/video_config.ini', 'w') as configfile:
    config.write(configfile)

def get_video(base_path):
  config = configparser.ConfigParser()
  config.read(base_path+'/video_config.ini')
  curr_vid = config['General']['curr_vid']
  return curr_vid

def get_prompts_video(base_path):
  config = configparser.ConfigParser()
  config.read(base_path+'/prompt_config.ini')
  curr_vid = config['General']['curr_vid']
  return curr_vid

In [ ]:
get_video(base_path)

'6'

In [ ]:

import cv2
import numpy as np


def align_text_to_image(text, image, font, font_scale, color, thickness, alignment = "Center",X_pos = None,Y_pos = None):
  if alignment == "Center":
    text_size, _ = cv2.getTextSize(text, font, font_scale, thickness)
    if X_pos == None:
      text_x = (image.shape[1] - text_size[0]) // 2
    if Y_pos == None:
      text_y = (image.shape[0] + text_size[1]) // 2
    cv2.putText(image, text, (text_x, text_y), font, font_scale, color, thickness)

def add_text_to_image(image, text, position=(330, 800), font=cv2.FONT_HERSHEY_COMPLEX, font_scale=3, color=(255, 255, 255), thickness=5):
  cv2.putText(image, text, position, font, font_scale, color, thickness)

def calculate_fps(text,vid_length):
  frames = len(text.split(" "))
  fps = frames / vid_length
  return fps

def textOnImg_vid(text,img,name="gg",frame_shape = None,title=None,title_align = "Center",font = cv2.FONT_HERSHEY_COMPLEX,vid_len=None,audio=None):

  video_name = f'{name}.mp4'
  if vid_len ==None:
    fps=24
  else:
    fps = calculate_fps(text,vid_len)
  frame = img
  if frame_shape == None:
    frame_shape = frame.shape
  height, width, layers = frame_shape
  img = cv2.resize(img, (width, height))


  if title !=None:
    if title_align == "Center":
      textsize = cv2.getTextSize(title, font, 1, 2)[0]
      textX = (width - textsize[0]) / 2
      textY = 150
      cv2.putText(img, title, (int(textX), int(textY)), font, 1, (0, 0, 0), 8)
      cv2.putText(img, title, (int(textX), int(textY)), font, 1, (255, 255, 255), 2)


  text_list = text.split(" ")
  running_text = ""
  font_scale = 1
  color = (255, 255, 255)
  thickness = 2
  font = cv2.FONT_HERSHEY_COMPLEX

  #video = cv2.VideoWriter(video_name, cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))

  frames = []

  for i,txt in enumerate(text_list):
    img1 = img.copy()
    running_text += txt + " "
    text_size, _ = cv2.getTextSize(running_text, font, font_scale, thickness)
    if text_size[0] > width:
      running_text = txt + " "

    align_text_to_image(running_text, img1, font, font_scale, (0,0,0), thickness*3, alignment = "Center",X_pos = None,Y_pos = None)
    align_text_to_image(running_text, img1, font, font_scale, color, thickness, alignment = "Center",X_pos = None,Y_pos = None)

    frames.append(img1)

  tot_frames = len(frames)

  # fps = tot_frames / vid_length
  # print("TOTAL FRAMES : ",tot_frames)
  # print("FPS : ",fps)
  # print("VIDEO LENGTH : ",vid_length)
  video = cv2.VideoWriter(video_name, cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))
  for imgx in frames:
    video.write(imgx)

  # Release the video writer
  video.release()
  cv2.destroyAllWindows()
  video1 = mp.VideoFileClip(video_name)
  video1 = video1.set_audio(audio)
  video1.write_videofile(f"{name}_final.mp4")


Auto_gen = True
if Auto_gen == True:
  while int(get_video(base_path)) < int(get_prompts_video(base_path)):
    set_video(base_path,int(get_video(base_path))+1)
    curr_vid = int(get_video(base_path))
    curr_vid_path = os.path.join(base_path,str(curr_vid))
    if not os.path.exists(f"{curr_vid_path}/video"):
      os.mkdir(f"{curr_vid_path}/video")
    with open(f"{curr_vid_path}/prompts.txt", "r") as f:
      responses = f.readlines()

    #textOnImg_vid(text,img,title="Why does it feel so good to scratch an itch?",title_align="Center",frame_shape=(1920,1080,3))
    final_video = []
    for i in range(len(responses)):
      import moviepy.editor as mp
      audio = mp.AudioFileClip(f"{curr_vid_path}/audio/{i+1}.wav")
      vid_length = audio.duration
      img = cv2.imread(f"{curr_vid_path}/images/{i+1}.png")
      textOnImg_vid(responses[i],img,name=f"{i}",title=responses[0],title_align="Center",frame_shape=(1920,1080,3),vid_len=vid_length,audio=audio)
      final_video.append(mp.VideoFileClip(f"{i}_final.mp4"))

    final_video = mp.concatenate_videoclips(final_video)
    final_video.write_videofile(f"{curr_vid_path}/video/final_video.mp4")




Moviepy - Building video 0_final.mp4.
MoviePy - Writing audio in 0_finalTEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video 0_final.mp4



t:  91%|█████████ | 10/11 [00:00<00:00, 37.42it/s, now=None]WARNING:py.warnings:/usr/local/lib/python3.10/dist-packages/moviepy/video/io/ffmpeg_reader.py:123: UserWarning: Warning: in file 0.mp4, 6220800 bytes wanted but 0 bytes read,at frame 10/11, at time 4.17/4.17 sec. Using the last valid frame instead.
  warnings.warn("Warning: in file %s, "%(self.filename)+



Moviepy - Done !
Moviepy - video ready 0_final.mp4
Moviepy - Building video 1_final.mp4.
MoviePy - Writing audio in 1_finalTEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video 1_final.mp4



...

t:  86%|████████▌ | 6/7 [00:00<00:00, 58.90it/s, now=None]WARNING:py.warnings:/usr/local/lib/python3.10/dist-packages/moviepy/video/io/ffmpeg_reader.py:123: UserWarning: Warning: in file 9.mp4, 6220800 bytes wanted but 0 bytes read,at frame 6/7, at time 5.45/5.47 sec. Using the last valid frame instead.
  warnings.warn("Warning: in file %s, "%(self.filename)+



Moviepy - Done !
Moviepy - video ready 9_final.mp4
Moviepy - Building video /content/drive/MyDrive/AI/18/video/final_video.mp4.
MoviePy - Writing audio in final_videoTEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video /content/drive/MyDrive/AI/18/video/final_video.mp4



t:  16%|█▌        | 23/142 [00:00<00:02, 46.31it/s, now=None]WARNING:py.warnings:/usr/local/lib/python3.10/dist-packages/moviepy/video/io/ffmpeg_reader.py:123: UserWarning: Warning: in file 1_final.mp4, 6220800 bytes wanted but 0 bytes read,at frame 10/11, at time 3.62/3.66 sec. Using the last valid frame instead.
  warnings.warn("Warning: in file %s, "%(self.filename)+



Moviepy - Done !
Moviepy - video ready /content/drive/MyDrive/AI/18/video/final_video.mp4
